# ST 生成器

- 总结 ST (Structured text) 语言的语法
- 自动化生成 ST 代码


## ST 语法

[ST 语法参考](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.49.2016&rep=rep1&type=pdf)


使用 openplc Editor 软件编写并生成的一段 ST 代码如下，该代码可导入到 openplc Runtime 中运行。

```iecst
PROGRAM program0
  VAR
    PB1 : BOOL;
    PB2 : BOOL;
    LED : BOOL;
  END_VAR

  IF PB1 THEN
    LED := TRUE;
  END_IF;

  IF PB2 THEN
    LED := FALSE;
  END_IF;
END_PROGRAM


CONFIGURATION Config0

  RESOURCE Res0 ON PLC
    TASK task0(INTERVAL := T#20ms,PRIORITY := 0);
    PROGRAM instance0 WITH task0 : program0;
  END_RESOURCE
END_CONFIGURATION
```
PROGRAM 部分是我们主要考虑的。


### 数据类型

https://en.wikipedia.org/wiki/IEC_61131-3



#### 基本数据类型

- 整数
- 浮点数
- 时间
- 字符串
- 布尔




#### 组合数据类型


- ARRAY
- STRUCT
- UNION
- Sub-range

```iecst
// 结构体
TYPE Rectangle :
    STRUCT
    TopLeft : Point;
    Height : INT;
    Width : INT;
    END_STRUCT;
END_TYPE

// 枚举
TYPE Color :
    (Red, White, Blue);
END_TYPE

// Sub-ranges
TYPE Angle :
    INT(-180..+180);
END_TYPE


// 数组
TYPE Display :
    ARRAY[1..768, 1..1024] OF Color;
END_TYPE

```




### 赋值语句 Assignments

```iecst
VarA := (VarB * 24 MOD 2 = 1) XOR (VarC <= VarD + 34);

(* AnArray is an array containing twenty integers *)
AnArray := 10(1), 5(2), 5(3);

VarA := AND(Var1, Var2, ..., VarN);
```
一些运算符可以有任何数量的参数（相同类型）


In [1]:
ST_ASSIGNMENTS_S = ["<变量名> := <计算表达式>;\n"]

### 函数调用语句 Function calls

*先不考虑*

注意，ST中不允许递归调用函数。

> Note that, strictly speaking, the FUNCTION … END_FUNCTION and the VAR … END_VAR constructs mentioned in this paragraph do not belong to the Structured text programming language, as discussed earlier.

```iecst
FUNCTION_BLOCK FB_Timed_Counter
    VAR_INPUT
        // ...
    END_VAR
    
    VAR_OUTPUT
        // ...
    END_VAR
    
    VAR
        // ...
    END_VAR
        
    // Start of Function Block programming
    
END_FUNCTION_BLOCK

// 另一种写法？
FUNCTION Ave_REAL : REAL
    VAR_INPUT
        Input1, Input2 : REAL;
    END_VAR
    Ave_REAL := (Input1 + Input2) / 2;
END_FUNCTION
Average1 := Ave_REAL(5.0, 4.0);
Average2 := Ave_REAL(Input2 := 6.0);
(* Value 3.0 assigned to Average2 *)


```


### 条件分支语句 Conditional statements

```iecst
IF VarB > 0 THEN
  IF VarC = 3 THEN
    VarA := TRUE;
  ELSE
    VarC := 3;
    VarA := FALSE;
  END_IF;
ELSIF VarB < 0 THEN
  VarA := FALSE;
ELSE
  VarC := 3;
  VarA := TRUE;
END_IF;


CASE Var233 OF
  0: 
  1:
ELSE
  
END_CASE;

```

In [2]:
ST_IF_S = ["""
IF <计算表达式> THEN  
    <程序块单元>+
(ELSIF <计算表达式> THEN
    <程序块单元>)+
(ELSE
    <程序块单元>)?
END_IF;
"""]

# TODO: 请补充完整
ST_CASE_S = ["""
CASE Var233 OF
  0: 
  1:
ELSE
  
END_CASE;
"""]

### 迭代循环语句 Iteration statements

3 种迭代循环语句

```iecst
FOR VarB := 1 TO VarC DO
  VarD := VarD + 1;
  VarE := 2 * VarE;
END_FOR;

FOR VarB := 100 TO 0 BY –2 DO
  VarD := VarD + VarB;
END_FOR;

WHILE VarA AND (VarD <= VarC) DO
  VarD := VarD + 1;
  VarE := 2 * VarE;
  VarA := VarE < 100;
END_WHILE;

REPEAT
  VarD := VarD + 1;
  VarE := 2 * VarE;
  VarA := VarE < 100;
UNTIL NOT(VarA AND (VarD <= VarC));

```


In [3]:
ST_FOR_S = ["""
FOR <变量名> TO <计算表达式> DO
  <程序块单元>+
END_FOR;
""",
"""
FOR <变量名> TO <计算表达式> BY <计算表达式> DO
  <程序块单元>+
END_FOR;
"""]

ST_WHILE_S = ["""
WHILE <计算表达式> DO
  <程序块单元>+
END_WHILE;
"""]

ST_REPEAT_S = ["""
REPEAT
  <程序块单元>+
UNTIL <计算表达式>;
"""]

## ST EBNF

In [4]:
from fuzzingbook.Grammars import EXPR_EBNF_GRAMMAR, srange, convert_ebnf_grammar, Grammar, Expansion, CHARACTERS_WITHOUT_QUOTE
from fuzzingbook.Grammars import is_valid_grammar, exp_string
import string


In [5]:
# tool: 通用表达式转换为字典
from fuzzingbook.Grammars import define_ex_grammar

@define_ex_grammar
def expression_grammar():
    number =    '' 
    integer_number =  ''+'233'

    
# expression_grammar
for symbol in expression_grammar:
    t = [f'<{i[0]}>' for i in expression_grammar[symbol]]
    print(f'"<{symbol}>" :\n    {t},')

"<number>" :
    ['<>'],
"<integer_number>" :
    ['<>'],


### 根据 PASCAL 试试看

> 方括号划界可选构建体；`{}` 表示封闭构造的零或更多重复；`()` 表示构造的简单分组；`|` 表示从许多人中选择一个；定义中的文字使用粗体字体或双引号标记表示。

> EBNF operators – `?` becomes (0,1), `*` becomes (0,), and `+` becomes (1,). 

```ebnf

```

In [6]:
LOW_LEVEL_EBNF: Grammar = {
    "<variable-list>":
        ["<variable>", "<variable-list> , <variable>"],
    "<identifier-list>":
        ["<identifier>", "<identifier-list> , <identifier>"],
    "<expression-list>":
        ["<expression>", "<expression-list> , <expression>"],
    "<number>":
        ['<integer_number>', '<real_number>'],
    "<integer_number>":
        ['<digit_sequence>'],
    "<real_number>":
        ['<digit_sequence>(.<digit_sequence>)?<scale_factor>?'],
    "<scale_factor>":
        ["E<sign>?<digit_sequence>","e<sign>?<digit_sequence>"],  
    "unsigned_digit_sequence":
        ["<digit>+"],
    "<digit_sequence>":
        ["<sign>?<unsigned_digit_sequence>"],
    "<sign>":
        [" +", " -"],
    "<letter>":
        srange(string.ascii_letters),
    "<digit>":
        srange(string.digits),
    "<string>":
        ["'<string_character>+'"],
    "<string_character>":
        srange(CHARACTERS_WITHOUT_QUOTE) + ["''"],  # ?
    "<label>":
        ["integer_number"],
    "<constant>":
        [""],  # TODO: [ sign ] (constant_identifier | number) | string
}

VARIABLE_IDENTIFIER_EBNF: Grammar = {
    "<identifier>": # TODO = letter { letter | digit } 
        ["<letter>+"],
    "<file-variable>":
        ["<variable>"],
    "<referenced-variable>":
        ["<pointer-variable>^"], # ?
    "<record-variable>":
        ["<variable>"],
    "<pointer-variable>":
        ["<variable>"],
    "<actual-variable>":
        ["<variable>"],
    "<array-variable>":
        ["<variable>"],
    "<field-identifier>":
        ["<identifier>"],
    "<constant-identifier>":
        ["<identifier>"],
    "<variable-identifier>":
        ["<identifier>"],
    "<type-identifier>":
        ["<identifier>"],
    "<procedure-identifier>":
        ["<identifier>"],
    "<function-identifier>":
        ["<identifier>"],
    "<bound-identifier>":
        ["<identifier>"],
}

# INPUT_OUTPUT_EBNF 
# Record Fields
# Types

PASCAL_EBNF_GRAMMAR: Grammar = {
    "<start>":
        ["<variable-list>"],
    **LOW_LEVEL_EBNF,
}
# assert is_valid_grammar(PASCAL_EBNF_GRAMMAR)
pascal_grammar = convert_ebnf_grammar(PASCAL_EBNF_GRAMMAR)
# assert is_valid_grammar(pascal_grammar)


### ST 语言 EBNF 自顶而下构建

1. 先生成变量列表
2. 根据变量列表生成程序主体

*时间有限，此处构建的仅为 ST 语言的子集*

范式：

```iecst
PROGRAM program0
  VAR
    // 生成变量列表
  END_VAR
  // 生成程序主体
END_PROGRAM


CONFIGURATION Config0

  RESOURCE Res0 ON PLC
    TASK task0(INTERVAL := T#20ms,PRIORITY := 0);
    PROGRAM instance0 WITH task0 : program0;
  END_RESOURCE
END_CONFIGURATION
```


#### 变量列表 EBNF

In [7]:
ST_VAR_EBNF_GRAMMAR: Grammar = {
    "<start>":
        ["<变量声明语句>+"],
    "<变量声明语句>":
        ["<variable-name> : <variable-type>;\n"],
    "<variable-name>": # 形如 Var233
        ["Var<digit>+"],
    "<variable-type>": # 形如 INT REAL
        ["INT","REAL"],
    "<digit>":
        srange(string.digits)
}

#### 程序主体 EBNF

In [8]:
ST_BODY_EBNF_GRAMMAR: Grammar = {
    "<start>":
        ["<程序块单元>+"],

    "<程序块单元>":
        ["<Assignments-statements>\n", # 赋值语句
        #  "<Function-statements>",  # 函数语句
         "<Conditional-statements>", # 条件语句
         "<Iteration-statements>"], # 循环语句

    "<Assignments-statements>":
        ST_ASSIGNMENTS_S,

    # "<Function-statements>": # 先不考虑函数
        # ["NIL;\n"],

    "<Conditional-statements>":
        ST_IF_S,

    "<Iteration-statements>":
        ST_FOR_S + ST_WHILE_S + ST_REPEAT_S,
        
    
    "<变量名>":
        ["Var233", "Var666", "Var42"], # TODO
        
    "<计算表达式>":
        ["<expr>"],

    # 基本算术表达式    
    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["<factor>", "(<expr>)", "<integer>(.<integer>)?", "<变量名>"],

    "<sign>":
        [" +", " -"],

    "<integer>":
        ["<sign>?<digit>+"],

    "<digit>":
        srange(string.digits)
}
assert is_valid_grammar(ST_BODY_EBNF_GRAMMAR)

## ST 生成

`GrammarCoverageFuzzer`

> Subsequent calls to `fuzz()` will go for further coverage (i.e., covering the other area code digits, for example); a call to `reset()` clears the recored coverage, starting anew.

In [9]:
from fuzzingbook.GrammarFuzzer import EvenFasterGrammarFuzzer
from fuzzingbook.GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [20]:
fbody = GrammarCoverageFuzzer(convert_ebnf_grammar(ST_BODY_EBNF_GRAMMAR),start_symbol="<start>",max_nonterminals=10)
body_s = fbody.fuzz()
print(body_s)


FOR Var666 TO Var42 + Var666 * Var666 / Var233 - Var233 BY Var233 DO
  
IF Var233 THEN  
    Var666 := Var233;


ELSIF Var233 THEN
    Var666 := Var233;



END_IF;

END_FOR;



In [12]:
fvar = GrammarCoverageFuzzer(convert_ebnf_grammar(ST_VAR_EBNF_GRAMMAR))
var_s = fvar.fuzz()


In [38]:


# 块拼接
MODEL_STR = f"""
PROGRAM program0
  VAR
    Var233 : BOOL;
    Var666 : BOOL;
    Var42 : BOOL;
  END_VAR
  
{body_s}

END_PROGRAM;

"""

# 写入文件
with open("st_gen.st", "w") as f:
    f.write(MODEL_STR)

## 测试

In [22]:
import subprocess
program = "/home/hxn/下载/OpenPLC_Editor/matiec/iec2c"
FILE = "st_gen.st"
arg = [program,
       "-f",
       "-l",
       "-p",
       "-I", "/home/hxn/下载/OpenPLC_Editor/matiec/lib",
       "-T", "/home/hxn/桌面/ST_study/openPLC_project_test/build",
       FILE]
# -f // display full token location on error messages
# -l // use a relaxed datatype equivalence model       (a non-standard extension?)
# -p // allow use of forward references                (a non-standard extension?)
# -I "/home/hxn/下载/OpenPLC_Editor/matiec/lib"           // include_directory
# -T "/home/hxn/桌面/ST_study/openPLC_project_test/build" // target_directory
# "/home/hxn/桌面/ST_study/openPLC_project_test/build/plc.st"
result = subprocess.run(arg,
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7

print(">>>stdout")
print(result.stdout)
print(">>>stderr")
print(result.stderr)
print(">>>returncode")
print(result.returncode)


>>>stdout

>>>stderr
st_gen.st:10-12..14-4: error: expecting ':=' between control variable and start expression in ST 'FOR' statement.
st_gen.st:20-2..21-10: error: invalid variable before ':=' in ST assignment statement.
st_gen.st:25-1..25-6: error: invalid statement in ST statement.
st_gen.st:27-1..27-7: error: invalid statement in ST statement.

5 error(s) found. Bailing out!

>>>returncode
1


## 参考资料

[EBNF for Pascal](http://www.cs.kent.edu/~durand/CS43101Fall2004/resources/Pascal-EBNF.html)